## 14.5 Example 1: Dementia

Let us describe the interpretation of the logistic model parameters through an example from the dementia dataset. In this example, we are interested to model the dementia status $Y_i$ of the subjects according to their sex. The logistic regression model that allows us to do so is

$$\mathrm{logit}(\pi_i) = \beta_0 + \beta_1sex_i$$

where $\pi_i=E(Y_i|sex_i)$. In R, the function used to estimate such a model is `glm` that stands for *generalized linear model*. The logistic regression model is a specific class of generalized linear model whose particularity is to have the logit function as a link function. To use the `glm` function, we need to give some important arguments:

* An `R` object called a formula that states the model we want to estimate. The general form of this formula is *y ~ x1 + x2 + ... + xp* for a model with outcome $y$ and $p$ explanatory variables $x_k$. Note that we could have included the intercept in the model by simply adding $1$ as explanatory variable *y ~ 1 + x1 + x2 + ... + xp*. However note that by default, `R`will include an intercept in the model even if not specified in the formula. If you ever want to estimate a model without an intercept, you have to include `-1` in the formula.
* The dataset from which the model is estimated
* The family of the *generalized linear model*, here we specify that we are working with a binomial outcome with the *logit* link function. Other options exist but are out of the scope of this module.

Finally, the `R` command to estimate the logistic model above on the dementia dataset and to store its results into an object `dementia1` is

In [1]:
dementia1 <- glm(dementia ~ sex, data = dementia, family = binomial(link="logit"))

ERROR: Error in is.data.frame(data): object 'dementia' not found


Now, we can explore the results that have just been stored into `dementia1` using the function `summary`

In [5]:
summary(dementia1)


Call:
glm(formula = dementia ~ sex, family = binomial(link = "logit"), 
    data = dementia)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-0.2211  -0.2211  -0.1771  -0.1771   2.8855  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept) -4.14722    0.02439 -170.01   <2e-16 ***
sex          0.44771    0.03264   13.72   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 38333  on 199999  degrees of freedom
Residual deviance: 38143  on 199998  degrees of freedom
AIC: 38147

Number of Fisher Scoring iterations: 7


Let us go through this summary in details part by part and describe its contents:

* *Call:* this part of the summary gives us the call we made to the function `glm`. It can be useful when a doubt arises about which model has been estimated.
* *Deviance residuals:* this part gives a summary of the distribution of all the estimated residuals. Here, the deviance residuals are given, they are defined as the individual contributions to the residual deviance (see below). They can be retrieved using the function `residuals` with option `type = "deviance"`. Note that there also exists other type of residuals for generalized linear models.
* *Coefficients:* this part gives the results of the estimation of the parameters from the model, here in our case, $\beta_0$ and $\beta_1$. For each parameter $\beta_k$, we have 
  * the point estimate $\hat{\beta}_k$
  * the standard error of the estimation $se(\hat{\beta}_k)$
  * the z-value computed as the ratio of the point estimate and the standard error $\hat{\beta}_k/se(\hat{\beta}_k)$. It is actually the Wald statistics whose null hypothesis is $\beta_k=0$.
  * the p-value of the Wald test (the star symbols gives the intensity of the significance level)
* *Null and residual deviances:* The deviance of a model is a measure of the goodness-of-fit of this model. It is defined as $D = -2(\ell_e - \ell_s)$ where $\ell_e$ and $\ell_s$ are respectively the log-likelihood of the estimated model and the log-likelihood of the saturated model. The saturated model uses the maximum possible number of parameters without redundancies and is the one with the best possible fit. The null deviance is simply the deviance computed for the null model, i.e. the minimal model with only an intercept and without any covariate. A bad model will have a high deviance and, for a good model, we expect the deviance to decrease, meaning that we better describe the data. However, we should remain cautious as a null deviance would mean a model with possible overfitting. Note that when computing deviances of different models for the same dataset, the log-likelihood of the saturated model $\ell_s$ would be constant. This is why often, as in the `glm`function output, the deviance is given in a simplified form as $-2\ell_e$ by removing the constant lok-likelihood from the saturated model. The degrees of freedom are given on the same line and equals $n-1$ where $n$ is the sample size and $1$ is the number of parameters to estimate in the null model The residual deviance is the deviance for the estimated model.
* *AIC:* the Akaike information criterion quantifies model fit as a function of the likelihood and the number of parameters being estimated. It is defined as $AIC = 2k - 2\ell(\hat{\beta})$ where $k$ is the number of parameter in the model and $\ell(\hat{\beta})$ the log-likelihood of the model computed at the estimated parameter values $\hat{\beta}$. The AIC is mainly used as a way to compare different models. The best model, in the scale of the AIC, is the one with the lowest AIC. Note that sometimes, contrarily to the *glm* package, the AIC is computed as $AIC = -2k + 2\ell(\hat{\beta})$ in which case the best model would be the one with the highest AIC value. We can observe that the AIC is actually minus the sum of the deviance and twice the number of the parameters. By including the number of parameters, the AIC can penalize a model that have too many parameters and avoids the selection of overfitting models.
* *Number of Fisher Scoring iterations:* it gives the number of iterations that the optimisation algorithm used to estimate the model performed before convergence

In this summary, we have everything that we need to give an interpretation of the estimated model. However, as we have seen, in the logistic regression, the crude parameters do not have a straightforward interpretation and we prefer to compute and interpret their exponential.

In [6]:
exp(coefficients(dementia1))

(Intercept)         sex 
 0.01580834  1.56472021

Let us interpret these as we just have seen. First, as this is not a case control study, we can interpret $\exp({\hat{\beta}_0})/(1+exp({\hat{\beta}_0})) = 0.0156$ as an estimation of the prevalence of dementia among the unexposed subjects, here the men. Actually, we can double check this by computing it directly from the data:

In [7]:
library(tidyverse, quietly = TRUE)
dementia %>% group_by(sex, dementia) %>% count()

Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
Registered S3 method overwritten by 'rvest':
  method            from
  read_xml.response xml2
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.1       ✔ purrr   0.3.2  
✔ tibble  2.1.1       ✔ dplyr   0.8.0.1
✔ tidyr   0.8.3       ✔ stringr 1.4.0  
✔ readr   1.3.1       ✔ forcats 0.4.0  
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


sex,dementia,n
0,0,107981
0,1,1707
1,0,88132
1,1,2180


In [8]:
round(1707 / (107981 + 1707),4)

[1] 0.0156

Now, we have that $\exp({\hat{\beta}_1}) = 1.565$. That means that the odds-ratio of developing dementia in the women versus the men is of $1.565$ or equivalently that the probability of developing dementia is higher among the exposed, here the women.

From the summary, we can deduced other interesting informations. First, if we look at the $p$-value for the sex variable, we can see that it is very significative meaning that we clearly reject the null hypothesis of no sex association. By looking at the residual deviance, we can see that by just including one parameter, the sex association, the deviance decreases by $190$. Decreasing the deviance is a good sign as it means that we are improving the description of the data with the estimated model. However, let us note again that decreasing the deviance is not a goal *per se* as it could lead to overfitting.

To summarize, a synthetic interpration of the estimated model could be the following: there is a significative association between the variable *sex* and the dementia status and the odds of developing dementia are $1.565$ times higher for the women in our study.

> *Exercise:* write a code that computes the odds-ratio $95\%$ confidence interval directly from the `R` output